# Proximal Policy Optimization Implementation

Installing library dependencies

In [ ]:
!pip install gym
!pip install pyglet==1.2.4
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')

!pip install unrar
!unrar x Roms.rar

# !python -m atari_py.import_roms rars
!pip install "gym[atari,accept-rom-license]"
!pip install atari-py
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!apt-get install python-opengl -y
!apt install xvfb -y
!python -m atari_py.import_roms ROMS

!pip install -U "ray[tune]"  # installs Ray + dependencies for Ray Tune
!pip install -U "ray[rllib]"  # installs Ray + dependencies for Ray RLlib
!pip install -U "ray[serve]"
!pip install 'ray[rllib]'==1.6
!apt-get install -y xvfb x11-utils
# !pip install pyvirtualdisplay==0.2.*
!pip install tensorboard
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading OpenAI Gym Beam Rider environment:**

Space Invaders environment is loaded and tried to understand the environment by looking into the observation space and action space

In [ ]:
import gym
import random

In [ ]:
from pyvirtualdisplay import Display    #for virtual display
display = Display(visible=0, size=(1400, 900))
_ = display.start()

In [ ]:
env = gym.make('BeamRider-v0')

In [ ]:
import random, math
import numpy as np

from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog

from gym.spaces import Discrete,Box

import ray
import ray.rllib.agents.dqn as dqn
from ray.tune.logger import pretty_print

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

from ray.tune.schedulers import ASHAScheduler

In [ ]:
from ray.rllib.agents import ppo

In [ ]:
#Load tensorboard notebook extension
%load_ext tensorboard

In [ ]:
config = {
    "use_critic" : True,
    "kl_coeff" : 0.2, # initial coefficient for KL divergence
    "train_batch_size" :4000, 
    "env" : "SpaceInvaders-v0",
    "framework" : "torch",
    "model" : {
        "fcnet_hiddens" : [32,32], # number of hidden layers used
        "fcnet_activation" : "linear", # activation function used
              },
    "sgd_minibatch_size" : 256, # minibatch size within each epoch
    "num_sgd_iter" : 30, # number of epochs to execute per train batch
    "lr" : 5e-5, #stepsize of SGD
    "vf_loss_coeff" : 1.0, # coefficient of the value function loss
    "clip_param" : 0.3, #PPO clip parameter
    "vf_clip_param" : 10,
    "kl_target" : 0.01,
    "batch_mode" : "truncate_episodes"

        }

stop = {"training_iteration": 20}

ray.shutdown()

ray.init(   
    num_cpus =8,
    num_gpus = 1,
    include_dashboard = False,
    ignore_reinit_error =True,
    log_to_driver = False,    
)
# Asha scheduler is used implement early stopping
asha_scheduler = ASHAScheduler(
    #time_attr='training_iteration',
    metric='episode_reward_mean',
    mode='max',
    
    )

trainer = ppo.PPOTrainer(config = config, env = 'BeamRider-v0' )

avg_rewards_ppo = []
for i in range(20):
  result = trainer.train()
  print(result['episode_reward_mean'])
  avg_rewards_ppo.append(result['episode_reward_mean'])

  if i % 5 == 0:
      checkpoint= trainer.save()
      print("checkpoint save at", checkpoint)



/usr/local/lib/python3.7/dist-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2022-04-23 22:02:13,216	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-04-23 22:02:13,219	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-04-23 22:02:23,306	INFO trainable.py:109 -- Trainable.setup took 10.092 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-23 22:02:23,310	WARNING util.py:55 -- Install gputil for GPU system monitoring.


nan
checkpoint save at /root/ray_results/PPO_BeamRider-v0_2022-04-23_22-02-13dzjndf7k/checkpoint_000001/checkpoint-1
396.0
330.0
334.4
313.5
312.4
checkpoint save at /root/ray_results/PPO_BeamRider-v0_2022-04-23_22-02-13dzjndf7k/checkpoint_000006/checkpoint-6
297.0
323.7142857142857
327.25
331.29411764705884
332.2
checkpoint save at /root/ray_results/PPO_BeamRider-v0_2022-04-23_22-02-13dzjndf7k/checkpoint_000011/checkpoint-11
322.0
330.0
335.0769230769231
326.2068965517241
321.2
checkpoint save at /root/ray_results/PPO_BeamRider-v0_2022-04-23_22-02-13dzjndf7k/checkpoint_000016/checkpoint-16
324.0
324.8235294117647
339.77777777777777
340.42105263157896


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=((10,8)))
#plt.xlabel = ('episodes')
#plt.ylabel = ('average rewards')
#plt.title = ('average rewards accumulated for episodes')
plt.plot(avg_rewards_dqn,'b.',alpha = .4)
plt.show()

<Figure size 720x576 with 1 Axes>


The code reference : 
Lab 8 and 9

https://github.com/ray-project/ray/blob/master/rllib/agents/trainer.py
https://usermanual.wiki/m/d8984f2e85323f04b1b3e4597da5fc177cf7b2e13344b957445d9aa729dfc985.pdf